In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from urllib.parse import quote
import json
import os


BASE_URL = "https://www.theflipside.io"
ARCHIVE_URL = f"{BASE_URL}/archives"
SEARCH_URL = f"{BASE_URL}/search"
HEADERS = {"User-Agent": "Mozilla/5.0"}


def search_fallback_link_for_date(date_obj):
    search_query = f"{date_obj.strftime('%b')} {date_obj.day}, {date_obj.year}"
    encoded_query = quote(search_query)
    search_url = f"{SEARCH_URL}?query={encoded_query}"

    print(f"Searching fallback for: {search_query} - {search_url}")

    try:
        resp = requests.get(search_url, headers=HEADERS)
        if resp.status_code != 200:
            print(f"Failed to fetch search page: {resp.status_code}")
            return None

        soup = BeautifulSoup(resp.text, "html.parser")
        result = soup.select_one("div.search-result-item a.link")

        if result:
            text = result.text.strip()
            if ", " in text:
                try:
                    title, date_str = text.rsplit(" , ", 1)
                    result_date = datetime.strptime(date_str, "%b %d, %Y")
                    if result_date.date() == date_obj.date():
                        url = result['href']
                        print(f"Search matched: {title.strip()} - {url}")
                        return {
                            "date": date_obj.strftime("%Y-%m-%d"),
                            "title": title.strip(),
                            "url": url if url.startswith("http") else BASE_URL + url,
                            "found_by": "search"
                        }
                    else:
                        print(f"First result does not match target date ({result_date.date()} != {date_obj.date()})")
                except Exception as e:
                    print(f"Failed to parse result: {text}")
        else:
            print("No search results found.")

    except Exception as e:
        print(f"Exception during search: {e}")

    print("No fallback result for this date.")
    return None


def fetch_flipside_articles(start_date_str, end_date_str, output_filename="flipside_links_full.json"):
    output_path = os.path.join("/content/drive/My Drive", output_filename)
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    print("Fetching Flipside archives...")
    resp = requests.get(ARCHIVE_URL, headers=HEADERS)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, "html.parser")
    archive_entries = soup.find_all('a', class_='link-block-3 w-inline-block')


    archive_map = {}
    for entry in archive_entries:
        href = entry.get('href')
        date_parts = entry.find_all('h4', class_='heading-12')
        title_tag = entry.find('h3', class_='heading-13')

        if len(date_parts) == 2:
            try:
                date = datetime.strptime(f"{date_parts[0].text.strip()} {date_parts[1].text.strip()} 2025", "%B %d %Y")
                full_url = href if href.startswith("http") else BASE_URL + href
                archive_map[date.date()] = {
                    "date": date.strftime("%Y-%m-%d"),
                    "title": title_tag.text.strip() if title_tag else "",
                    "url": full_url,
                    "found_by": "archive"
                }
            except Exception as e:
                print(f"Error parsing archive entry: {e}")
                continue


    results = []
    current = start_date
    id_counter = 1

    print("\n Checking articles between:", start_date_str, "to", end_date_str)
    while current <= end_date:
        print(f"\n Processing date: {current.strftime('%Y-%m-%d')}")
        entry = archive_map.get(current.date())

        if not entry:
            entry = search_fallback_link_for_date(current)

        if entry:
            entry["id"] = id_counter
            results.append(entry)
            print(f"Added [{entry['found_by']}]: {entry['url']}")
            id_counter += 1
        else:
            print("No article found for this date.")

        current += timedelta(days=1)


    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"\n Done! Saved {len(results)} entries to: {output_path}")

fetch_flipside_articles("2019-01-01", "2020-12-31", "flipside_links_19-20.json")


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.parse import urljoin


input_json_path = "/content/drive/MyDrive/flipside_links_19-20.json"
output_json_path = "/content/drive/My Drive/flipside_articles_19-20.json"

HEADERS = {"User-Agent": "Mozilla/5.0"}



def extract_articles_from_section(blocks, side, headline):
    filtered_blocks = []
    for block in blocks:
        class_list = block.get("class", [])
        if "w-dyn-bind-empty" not in class_list:
            filtered_blocks.append(block)

    print(f"Found {len(filtered_blocks)} valid {side} blocks")

    articles = []
    if len(filtered_blocks) == 1:
        print(f"Single block in {side}, attempting to split by </strong>...")
        raw_html = str(filtered_blocks[0])
        segments = raw_html.split("</strong>")
        for idx, segment in enumerate(segments):
            seg_soup = BeautifulSoup(segment, "html.parser")
            text = seg_soup.get_text().strip()
            link_tag = seg_soup.find("a")
            if text and len(text)>10:
                print(f'##{text}##')
                articles.append({
                    "headline": headline,
                    "text": text,
                    "link": link_tag["href"] if link_tag else ""
                })
                print(f"{side.capitalize()}[{idx}] - {'link' if link_tag else 'no link'}")
        return articles

    for idx, block in enumerate(filtered_blocks):
        soup = BeautifulSoup(str(block), "html.parser")
        text = soup.get_text().strip()
        link_tag = soup.find("a")
        if text and len(text)>10:
            articles.append({
                "headline": headline,
                "text": text,
                "link": link_tag["href"] if link_tag else ""
            })
            print(f"{side.capitalize()}[{idx}] - {'link' if link_tag else 'no link'}")

    return articles




def parse_article_page(entry):
    url = entry["url"]
    print(f"\n Fetching: {url}")

    try:
        response = requests.get(url, headers=HEADERS)
        response.encoding = 'utf-8'

        if response.status_code != 200:
            print("Failed to fetch article")
            return None

        soup = BeautifulSoup(response.text, "html.parser")
        parsed_entry = {
            "id": entry["id"],
            "date": entry["date"],
            "source_url": entry["url"],
            "source_title": entry["title"],
            "found_by": entry["found_by"],
            "neutral": [],
            "left": [],
            "right": []
        }

        print("Parsing neutral section...")
        neutral_block = soup.find("div", class_="rich-text-block-2")
        if neutral_block:
            raw_html = str(neutral_block)
            sections = raw_html.split("</strong>")
            for idx, section in enumerate(sections):
                sec_soup = BeautifulSoup(section, "html.parser")
                link_tag = sec_soup.find("a")
                text = sec_soup.get_text().strip()
                if link_tag and text:
                    parsed_entry["neutral"].append({
                        "headline": entry["title"],
                        "text": text,
                        "link": link_tag["href"]
                    })
                    print(f"Neutral[{idx}] - {link_tag['href'][:50]}...")



        def get_valid_heading(soup, class_name):
          candidates = soup.find_all("h1", class_=class_name)
          for tag in candidates:
              if tag.get_text(strip=True) and "w-dyn-bind-empty" not in tag.get("class", []):
                  return tag
          return None


        left_column_heading = get_valid_heading(soup, "from-the-left-heading")
        right_column_heading = get_valid_heading(soup, "from-the-right-heading")

        left_label = left_column_heading.get_text(strip=True).split()[-1].lower() if left_column_heading else ""
        right_label = right_column_heading.get_text(strip=True).split()[-1].lower() if right_column_heading else ""

        is_flipped = left_label == "right" or right_label == "left"
        print(f"Left heading: '{left_label}', Right heading: '{right_label}' - Flipped: {is_flipped}")


        left_intro = soup.select_one("div.paragraph-6.left.intro")
        left_headline = left_intro.get_text(strip=True) if left_intro else ""

        right_intro = soup.select_one("div.paragraph-6.right.intro")
        right_headline = right_intro.get_text(strip=True) if right_intro else ""

        left_blocks = soup.select("div.paragraph-6.left.bullet")
        right_blocks = soup.select("div.paragraph-6.right.bullet")

        if is_flipped:
            print("Flipped detected. Swapping columns.")
            print("Parsing left section...")
            parsed_entry["left"] = extract_articles_from_section(right_blocks, "left", left_headline)
            print("Parsing right section...")
            parsed_entry["right"] = extract_articles_from_section(left_blocks, "right", right_headline)
        else:
            print("Parsing left section...")
            parsed_entry["left"] = extract_articles_from_section(left_blocks, "left", left_headline)
            print("Parsing right section...")
            parsed_entry["right"] = extract_articles_from_section(right_blocks, "right", right_headline)


        return parsed_entry

    except Exception as e:
        print(f"Exception while parsing: {e}")
        return None

with open(input_json_path, "r", encoding="utf-8") as f:
    input_data = json.load(f)

parsed_results = []

for entry in input_data:
    result = parse_article_page(entry)
    if result:
        parsed_results.append(result)

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(parsed_results, f, indent=2, ensure_ascii=False)

print(f"\n Done! Parsed {len(parsed_results)} articles and saved to: {output_json_path}")


In [ ]:
import json
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def fetch_article_data(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f"Skipped - {url} (HTTP {response.status_code})")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string.strip() if soup.title else ""

        paragraphs = soup.find_all('p')
        body = "\n".join(p.get_text().strip() for p in paragraphs if p.get_text(strip=True))

        if len(body) < 300:
            print(f"Skipped - {url} (Too short: {len(body)} chars)")
            return None

        return {
            "headline": title,
            "text": body
        }

    except Exception as e:
        print(f"Exception fetching {url} - {str(e).splitlines()[0]}")
        return None

def process_entry(entry):
    print(f"\nProcessing ID {entry['id']} — {entry['source_title']}")
    output_entry = {
        "id": entry["id"],
        "date": entry["date"],
        "source_url": entry["source_url"],
        "source_title": entry["source_title"],
        "found_by": entry["found_by"],
        "neutral": [],
        "left": [],
        "right": []
    }

    for side in ["neutral", "left", "right"]:
        print(f"Fetching original {side} articles...")
        for art in entry.get(side, []):
            url = art.get("link")
            if not url:
                print(f"Skipped - missing link in {side} article")
                continue

            fetched = fetch_article_data(url)
            if fetched:
                output_entry[side].append({
                    "headline": fetched["headline"],
                    "text": fetched["text"],
                    "link": url
                })

    return output_entry

INPUT_PATH = "/content/drive/MyDrive/flipside_articles_20-21.json"
OUTPUT_PATH = "/content/drive/MyDrive/flipside_full_articles_20-21.json"

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

final_output = []

for idx, entry in enumerate(data):
    parsed = process_entry(entry)

    if any(parsed[section] for section in ["neutral", "left", "right"]):
        final_output.append(parsed)

        if idx % 5 == 0:
            with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
                json.dump(final_output, f, indent=2, ensure_ascii=False)
            print(f"Backup saved at {OUTPUT_PATH} after {idx+1} entries")

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(final_output, f, indent=2, ensure_ascii=False)

print(f"\n Done! Saved {len(final_output)} filtered entries to: {OUTPUT_PATH}")


In [ ]:
!pip install newspaper4k lxml_html_clean
!python -m nltk.downloader punkt


import json
import os
from newspaper import Article

INPUT_PATH = "/content/drive/MyDrive/flipside_articles_21-22.json"
OUTPUT_PATH = "/content/drive/MyDrive/flipside_4k_articles_21-22.json"


def fetch_article_data(url):
    try:
        article = Article(url, language='en')
        article.download()
        article.parse()

        title = article.title.strip() if article.title else ""
        text = article.text.strip()

        if len(text) < 300:
            print(f"Skipping {url} (Too short: {len(text)} chars)")
            return None

        return {
            "headline": title,
            "text": text
        }

    except Exception as e:
        print(f"Error fetching {url} - {e}")
        return None


def process_entry(entry):
    print(f"\n Processing ID {entry['id']} — {entry['source_title']}")
    output_entry = {
        "id": entry["id"],
        "date": entry["date"],
        "source_url": entry["source_url"],
        "source_title": entry["source_title"],
        "found_by": entry["found_by"],
        "neutral": [],
        "left": [],
        "right": []
    }

    for side in ["neutral", "left", "right"]:
        print(f"Fetching {side} articles...")
        for art in entry.get(side, []):
            url = art.get("link")
            if not url:
                print(f"Missing link in {side}")
                continue

            fetched = fetch_article_data(url)
            if fetched:
                output_entry[side].append({
                    "headline": fetched["headline"],
                    "text": fetched["text"],
                    "link": url
                })

    return output_entry


with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

final_output = []

for idx, entry in enumerate(data):
    parsed = process_entry(entry)

    if any(parsed[section] for section in ["neutral", "left", "right"]):
        final_output.append(parsed)

        if (idx + 1) % 5 == 0:
            with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
                json.dump(final_output, f, indent=2, ensure_ascii=False)
            print(f"Backup saved after {idx + 1} entries - {OUTPUT_PATH}")

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(final_output, f, indent=2, ensure_ascii=False)

print(f"\n Done! Saved {len(final_output)} entries with valid articles to: {OUTPUT_PATH}")


In [ ]:
import json

OUTPUT_PATH = "/content/drive/MyDrive/flipside_4k_articles_23-25.json"

with open(OUTPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

complete_count = 0
for entry in data:
    if all(entry.get(section) for section in ["neutral", "left", "right"]):
        complete_count += 1

print(f"Total entries with all 3 sides: {complete_count} out of {len(data)}")


In [ ]:
# TEST
import json
import os
import nltk
import torch
import evaluate
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

nltk.download('punkt')

# Load model/tokenizer
model_name = "allenai/led-base-16384"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"\n✅ Using device: {device}\n")

# JSON file paths
json_files = ["/kaggle/input/flipside-articles/flipside_4K_articles_23-25.json", "/kaggle/input/flipside-articles/flipside_4k_articles_21-22.json", "/kaggle/input/flipside-articles/flipside_4k_articles_19-20.json"]


triplets = []
input_lengths = []
target_lengths = []
total_lengths = []

# Load and process triplets
for json_file in json_files:
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for entry in data:
            left_text = " ".join([a['text'] for a in entry.get("left", [])])
            right_text = " ".join([a['text'] for a in entry.get("right", [])])
            neutral_text = " ".join([a['text'] for a in entry.get("neutral", [])])

            if left_text.strip() and right_text.strip() and neutral_text.strip():
                input_text = f"Left:\n{left_text}\n\nRight:\n{right_text}"
                input_tokens = tokenizer(input_text, truncation=True, max_length=16384)
                target_tokens = tokenizer(neutral_text, truncation=True, max_length=16384)

                input_len = len(input_tokens["input_ids"])
                target_len = len(target_tokens["input_ids"])

                input_lengths.append(input_len)
                target_lengths.append(target_len)
                total_lengths.append(input_len + target_len)

                triplets.append((input_text, neutral_text))

# Average lengths
print("📏 Token Length Stats:")
print(f"Input (Left+Right): {sum(input_lengths)/len(input_lengths):.2f} tokens (avg)")
print(f"Target (Neutral):   {sum(target_lengths)/len(target_lengths):.2f} tokens (avg)")
print(f"Total (Combined):   {sum(total_lengths)/len(total_lengths):.2f} tokens (avg)")

# 📊 Plot distributions
plt.figure(figsize=(15, 4))
plt.subplot(1, 3, 1)
plt.hist(input_lengths, bins=30, color='dodgerblue', edgecolor='black')
plt.title("Input Token Lengths (Left + Right)")
plt.xlabel("Tokens")
plt.ylabel("Count")

plt.subplot(1, 3, 2)
plt.hist(target_lengths, bins=30, color='mediumseagreen', edgecolor='black')
plt.title("Target Token Lengths (Neutral)")
plt.xlabel("Tokens")

plt.subplot(1, 3, 3)
plt.hist(total_lengths, bins=30, color='salmon', edgecolor='black')
plt.title("Total Token Lengths (Input + Target)")
plt.xlabel("Tokens")

plt.tight_layout()
plt.show()



idx = 0
input_text, target_text = triplets[idx]
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=16384)
print("Token length:", len(inputs["input_ids"][0]))
print("Sample input:\n", input_text[:500])  # Preview first 500 chars

import time

generated_summaries = []
references = []

max_gen_len = 1024         # Max length of generated summary
max_input_len = 16000      # Slightly under LED limit to be safe
skipped_count = 0

print("⚙️ Running inference on first 10 samples...")

for idx, (input_text, target_text) in enumerate(tqdm(triplets[:10])):
    try:
        # Tokenize input (left + right)
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=max_input_len
        )

        token_len = inputs["input_ids"].shape[1]
        print(f"[{idx}] Input token length: {token_len}")

        # Skip empty or malformed
        if token_len == 0:
            print(f"⚠️ Skipping empty input at index {idx}")
            skipped_count += 1
            continue

        # Move to device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        global_attention_mask = torch.zeros_like(inputs["input_ids"]).to(device)
        global_attention_mask[:, 0] = 1  # Only first token has global attention

        # Generate
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                global_attention_mask=global_attention_mask,
                max_length=max_gen_len,
                num_beams=4,
                no_repeat_ngram_size=3,
                early_stopping=True
            )

        # Decode and save
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_summaries.append(generated_text)
        references.append(target_text)

        # Clear GPU memory
        torch.cuda.empty_cache()
        time.sleep(0.2)  # Optional cooldown

    except Exception as e:
        print(f"❌ FAILED at index {idx}: {e}")
        skipped_count += 1
        torch.cuda.empty_cache()
        continue

# Final status
print(f"\n✅ Inference done.")
print(f"🔢 Total attempted: {len(triplets[:100])}")
print(f"✅ Generated: {len(generated_summaries)}")
print(f"⚠️ Skipped: {skipped_count}")


import nltk
import evaluate

# 1. Ensure metrics are installed
bertscore = evaluate.load("bertscore")
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# 2. BERTScore and ROUGE expect plain strings
assert all(isinstance(p, str) for p in generated_summaries)
assert all(isinstance(r, str) for r in references)

# ✅ Evaluate BERTScore
bertscore_result = bertscore.compute(predictions=generated_summaries, references=references, lang="en")

# ✅ Evaluate ROUGE
rouge_result = rouge.compute(predictions=generated_summaries, references=references)

# 3. BLEU requires tokenized input (double tokenized for refs)
bleu_preds = [nltk.word_tokenize(p) for p in generated_summaries]
bleu_refs = [[nltk.word_tokenize(r)] for r in references]  # <- list of list of tokens

# ✅ Evaluate BLEU
bleu_result = bleu.compute(predictions=bleu_preds, references=bleu_refs)

# 4. Print results
print("\n🔍 Evaluation Metrics:")
print(f"BERTScore - Precision: {sum(bertscore_result['precision']) / len(bertscore_result['precision']):.4f}")
print(f"BERTScore - Recall:    {sum(bertscore_result['recall']) / len(bertscore_result['recall']):.4f}")
print(f"BERTScore - F1:        {sum(bertscore_result['f1']) / len(bertscore_result['f1']):.4f}")
print(f"ROUGE: {rouge_result}")
print(f"BLEU:  {bleu_result['bleu']:.4f}")
